In [10]:
import numpy as np
import pandas as pd
from glob import glob
import xml.etree.ElementTree as xet
import cv2
import os

In [11]:
df = pd.read_csv('lables.csv')
df.head()

,filepath,xmin,xmax,ymin,ymax
0,images_labeled/N33.xml,856,1218,665,751
1,images_labeled/N204.xml,94,206,307,334
2,images_labeled/N105.xml,360,434,174,195
3,images_labeled/N239.xml,106,263,238,276
4,images_labeled/N201.xml,157,290,193,229


#### labels for yolo
center_x, center_y, width, height

In [52]:
# parsing
def parsing(path):
    parser = xet.parse(path).getroot()
    name  = parser.find('filename').text
    filename = f'./images_labeled/{name}'
    
    # width and height
    parser_size = parser.find('size')
    width = int(parser_size.find('width').text)
    height = int(parser_size.find('height').text)

    return filename, width, height
df[['filename', 'width', 'height']] = df['filepath'].apply(parsing).apply(pd.Series)

In [53]:
df.head()

,filepath,xmin,xmax,ymin,ymax,filename,width,height,center_x,center_y,bb_width,bb_height
0,images_labeled/N33.xml,856,1218,665,751,./images_labeled/N33.jpeg,1920,1080,0.540104,0.655556,0.188542,0.079630
1,images_labeled/N204.xml,94,206,307,334,./images_labeled/N204.jpeg,736,483,0.203804,0.663561,0.152174,0.055901
2,images_labeled/N105.xml,360,434,174,195,./images_labeled/N105.jpeg,500,289,0.794000,0.638408,0.148000,0.072664
3,images_labeled/N239.xml,106,263,238,276,./images_labeled/N239.jpeg,400,374,0.461250,0.687166,0.392500,0.101604
4,images_labeled/N201.xml,157,290,193,229,./images_labeled/N201.jpeg,700,466,0.319286,0.452790,0.190000,0.077253


In [54]:
# center_x, center_y, width, height
df['center_x'] = (df['xmax'] + df['xmin'])/(2*df['width'])
df['center_y'] = (df['ymax'] + df['ymin'])/(2*df['height'])

df['bb_width'] = (df['xmax'] - df['xmin'])/(df['width'])
df['bb_height'] = (df['ymax'] - df['ymin'])/(df['height'])

In [55]:
df.head()

,filepath,xmin,xmax,ymin,ymax,filename,width,height,center_x,center_y,bb_width,bb_height
0,images_labeled/N33.xml,856,1218,665,751,./images_labeled/N33.jpeg,1920,1080,0.540104,0.655556,0.188542,0.079630
1,images_labeled/N204.xml,94,206,307,334,./images_labeled/N204.jpeg,736,483,0.203804,0.663561,0.152174,0.055901
2,images_labeled/N105.xml,360,434,174,195,./images_labeled/N105.jpeg,500,289,0.794000,0.638408,0.148000,0.072664
3,images_labeled/N239.xml,106,263,238,276,./images_labeled/N239.jpeg,400,374,0.461250,0.687166,0.392500,0.101604
4,images_labeled/N201.xml,157,290,193,229,./images_labeled/N201.jpeg,700,466,0.319286,0.452790,0.190000,0.077253


In [56]:
#### split the data into train and test
df.shape

(225, 12)

In [57]:
df_train = df.iloc[:200]
df_test = df.iloc[200:]
print(df_train)

                    filepath  xmin  xmax  ymin  ymax  \
0     images_labeled/N33.xml   856  1218   665   751   
1    images_labeled/N204.xml    94   206   307   334   
2    images_labeled/N105.xml   360   434   174   195   
3    images_labeled/N239.xml   106   263   238   276   
4    images_labeled/N201.xml   157   290   193   229   
..                       ...   ...   ...   ...   ...   
195  images_labeled/N138.xml    36   465    64   168   
196  images_labeled/N149.xml   186   324   116   148   
197  images_labeled/N128.xml    58   221   198   271   
198  images_labeled/N241.xml   251   542   308   385   
199  images_labeled/N111.xml    80   239   364   402   

                       filename  width  height  center_x  center_y  bb_width  \
0     ./images_labeled/N33.jpeg   1920    1080  0.540104  0.655556  0.188542   
1    ./images_labeled/N204.jpeg    736     483  0.203804  0.663561  0.152174   
2    ./images_labeled/N105.jpeg    500     289  0.794000  0.638408  0.148000   
3    ./

In [58]:
### text
### class_id, center_x, center_y, bb_width, bb_height
from shutil import copy

In [59]:
train_folder = './data_images/train'
# copy each image into the folder
values = df_train[['filename', 'center_x', 'center_y', 'bb_width', 'bb_height']].values
for fname , x, y, w, h in values:
    image_name = os.path.split(fname)[-1]
    txt_name = os.path.splitext(image_name)[0]

    dst_image_path = os.path.join(train_folder, image_name)
    dst_label_file = os.path.join(train_folder, txt_name + '.txt')
    copy(fname, dst_image_path)
    # copy each image into the folder

    label_txt = f'0 {x} {y} {w} {h}'
    with open(dst_label_file,mode='w') as f:
        f.write(label_txt)

        f.close()
# generate .txt file which has label info

In [60]:
test_folder = './data_images/test'
# copy each image into the folder
values = df_test[['filename', 'center_x', 'center_y', 'bb_width', 'bb_height']].values
for fname , x, y, w, h in values:
    image_name = os.path.split(fname)[-1]
    txt_name = os.path.splitext(image_name)[0]

    dst_image_path = os.path.join(test_folder, image_name)
    dst_label_file = os.path.join(test_folder, txt_name + '.txt')
    copy(fname, dst_image_path)
    # copy each image into the folder

    label_txt = f'0 {x} {y} {w} {h}'
    with open(dst_label_file,mode='w') as f:
        f.write(label_txt)

        f.close()
# generate .txt file which has label info